# Impulse Current Calculator - LITE Version

## Quick Usage Guide

**Single universal function for all impulse types:**

```python
run_impulse_calculation(
    impulse_type='PEB',        # 'PEB', 'NEB', 'NFB', or '8/20'
    function_type='heidler',   # 'heidler', 'double_exp', 'both', or 'damped_sine' (for 8/20)
    I_peak=200e3,              # Peak current in A (e.g., 200e3 = 200 kA)
    duration='infinity',       # 'infinity' = auto-calculate, or specify in seconds
    dt=1e-8,                   # Time step in seconds (1e-8 = 10 ns)
    show_derivative=False,     # True = show di/dt plot
    compare_functions_flag=False  # True = compare Heidler vs Double-Exp
)
```

**Parameters:**
- `impulse_type`: Impulse designation (PEB=10/350µs, NEB=1/200µs, NFB=0.25/100µs, 8/20=8/20µs)
- `function_type`: Waveform model (Heidler or double-exponential; 8/20 uses damped_sine)
- `I_peak`: Peak current value (if None, uses IEC reference for LPL I)
- `duration`: Simulation time ('infinity' recommended for accurate W/R and Q)
- `dt`: Time step (smaller = more accurate, but slower)
- `show_derivative`: Add di/dt plot to visualize current steepness
- `compare_functions_flag`: Plot both Heidler and double-exp side-by-side

**Output:** Plots + calculated IEC parameters (W/R, Q, di/dt_max)

## Setup (run once)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Parameter database from my dissertation (Table D.1 and D.2)
IMPULSE_PARAMS = {
    'PEB': {
        'name': 'Positive First Stroke (PEB)', 'designation': '10/350 µs',
        'heidler': {'tau1': 18.8e-6, 'tau2': 485e-6, 'eta': 0.93, 'n': 10},
        'double_exp': {'tau1': 4.064e-6, 'tau2': 470.107e-6, 'eta': 0.951}
    },
    'NEB': {
        'name': 'Negative First Stroke (NEB)', 'designation': '1/200 µs',
        'heidler': {'tau1': 1.826e-6, 'tau2': 285e-6, 'eta': 0.988, 'n': 10},
        'double_exp': {'tau1': 0.374e-6, 'tau2': 284.328e-6, 'eta': 0.99}
    },
    'NFB': {
        'name': 'Negative Subsequent Stroke (NFB)', 'designation': '0.25/100 µs',
        'heidler': {'tau1': 0.454e-6, 'tau2': 143.4e-6, 'eta': 0.993, 'n': 10},
        'double_exp': {'tau1': 0.092e-6, 'tau2': 143.134e-6, 'eta': 0.995}
    },
    '8/20': {
        'name': 'Short Impulse (8/20 µs)', 'designation': '8/20 µs',
        'damped_sine': {'tau': 24e-6, 'omega': 120023, 'eta': 0.615}
    }
}

REFERENCE_VALUES = {
    'PEB': {'I_peak': 200e3, 'Q': 100, 'W_R': 10e6},
    'NEB': {'I_peak': 100e3, 'Q': 28.7, 'W_R': 1.44e6},
    'NFB': {'I_peak': 50e3, 'Q': 7.2, 'W_R': 0.18e6},
    '8/20': {'I_peak': 10e3, 'Q': 5, 'W_R': 0.05e6}
}

def heidler_function(t, I_peak, tau1, tau2, eta, n=10):
    """Heidler function: i(t) = (I/eta) * (t/tau1)^n / (1 + (t/tau1)^n) * exp(-t/tau2)"""
    t_ratio = t / tau1
    current = (I_peak / eta) * (t_ratio**n / (1 + t_ratio**n)) * np.exp(-t / tau2)
    return current * (I_peak / np.max(current))  # Normalize to exact peak

def double_exp_function(t, I_peak, tau1, tau2, eta):
    """Double-exponential: i(t) = (I/eta) * (exp(-t/tau2) - exp(-t/tau1))"""
    current = (I_peak / eta) * (np.exp(-t / tau2) - np.exp(-t / tau1))
    return current * (I_peak / np.max(current))

def damped_sine_function(t, I_peak, tau, omega, eta):
    """Damped sine wave (8/20): i(t) = (I/eta) * exp(-t/tau) * sin(omega*t)"""
    current = (I_peak / eta) * np.exp(-t / tau) * np.sin(omega * t)
    return current * (I_peak / np.max(current))

def calculate_parameters(t, i, impulse_type=None):
    """Calculate IEC parameters: W/R, Q, di/dt_max"""
    use_abs = (impulse_type == '8/20')  # For bipolar waveforms
    return {
        'I_peak': np.max(i), 'I_peak_kA': np.max(i)/1e3, 't_peak': t[np.argmax(i)],
        'W_R': np.trapezoid(i**2, t), 'W_R_MJ': np.trapezoid(i**2, t)/1e6,
        'Q': np.trapezoid(np.abs(i) if use_abs else i, t),
        'di_dt_max': np.max(np.gradient(i, t)), 'di_dt_max_kA_us': np.max(np.gradient(i, t))/1e9
    }

def plot_waveform(t, i, title, params, show_derivative=False):
    """Plot current waveform with parameters"""
    fig, axes = plt.subplots(2 if show_derivative else 1, 1, figsize=(12, 8 if show_derivative else 5))
    ax1 = axes[0] if show_derivative else axes
    
    t_us, i_kA = t * 1e6, i / 1e3
    ax1.plot(t_us, i_kA, 'b-', linewidth=2)
    ax1.set_ylabel('Current (kA)', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Time (µs)', fontsize=12, fontweight='bold')
    ax1.set_title(title, fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Add parameter box
    energy_str = f"{params['W_R_MJ']:.3f} MJ/Ω" if params['W_R_MJ'] >= 1 else f"{params['W_R']/1e3:.2f} kJ/Ω"
    textstr = f"Peak: {params['I_peak_kA']:.2f} kA\\nEnergy: {energy_str}\\nCharge: {params['Q']:.2f} C\\ndi/dt: {params['di_dt_max_kA_us']:.1f} kA/µs"
    ax1.text(0.98, 0.97, textstr, transform=ax1.transAxes, fontsize=10,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    if show_derivative:
        di_dt = np.gradient(i, t) / 1e9  # kA/µs
        t_peak = params['t_peak']
        zoom_end = min(t_peak * 3, t[-1])
        zoom_idx = t <= zoom_end
        axes[1].plot(t_us[zoom_idx], di_dt[zoom_idx], 'g-', linewidth=2)
        axes[1].set_xlabel('Time (µs)', fontsize=12, fontweight='bold')
        axes[1].set_ylabel('di/dt (kA/µs)', fontsize=12, fontweight='bold')
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()

def run_impulse_calculation(impulse_type='PEB', function_type='heidler', I_peak=None,
                           duration='infinity', dt=1e-8, show_derivative=False,
                           compare_functions_flag=False):
    """Universal impulse current calculator"""
    if I_peak is None:
        I_peak = REFERENCE_VALUES[impulse_type]['I_peak']
    
    if duration == 'infinity':
        multiplier = 7 if impulse_type != '8/20' else 5
        tau_key = 'tau' if impulse_type == '8/20' else 'tau2'
        params_key = 'damped_sine' if impulse_type == '8/20' else 'heidler'
        duration = multiplier * IMPULSE_PARAMS[impulse_type][params_key][tau_key]
    
    t = np.arange(0, duration, dt)
    results = {}
    
    if impulse_type == '8/20':
        p = IMPULSE_PARAMS['8/20']['damped_sine']
        i = damped_sine_function(t, I_peak, p['tau'], p['omega'], p['eta'])
        results['damped_sine'] = calculate_parameters(t, i, impulse_type)
        plot_waveform(t, i, f"{IMPULSE_PARAMS[impulse_type]['name']}", results['damped_sine'], show_derivative)
    else:
        if function_type in ['heidler', 'both']:
            p = IMPULSE_PARAMS[impulse_type]['heidler']
            i_h = heidler_function(t, I_peak, p['tau1'], p['tau2'], p['eta'], p['n'])
            results['heidler'] = calculate_parameters(t, i_h, impulse_type)
            if not compare_functions_flag:
                plot_waveform(t, i_h, f"Heidler: {IMPULSE_PARAMS[impulse_type]['name']}",
                            results['heidler'], show_derivative)
        
        if function_type in ['double_exp', 'both']:
            p = IMPULSE_PARAMS[impulse_type]['double_exp']
            i_d = double_exp_function(t, I_peak, p['tau1'], p['tau2'], p['eta'])
            results['double_exp'] = calculate_parameters(t, i_d, impulse_type)
            if not compare_functions_flag:
                plot_waveform(t, i_d, f"Double-Exp: {IMPULSE_PARAMS[impulse_type]['name']}",
                            results['double_exp'], show_derivative)
        
        if compare_functions_flag and function_type == 'both':
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.plot(t*1e6, i_h/1e3, 'b-', linewidth=2, label='Heidler')
            ax.plot(t*1e6, i_d/1e3, 'r--', linewidth=2, label='Double-Exp')
            ax.set_xlabel('Time (µs)', fontsize=12, fontweight='bold')
            ax.set_ylabel('Current (kA)', fontsize=12, fontweight='bold')
            ax.set_title(f"Comparison: {IMPULSE_PARAMS[impulse_type]['name']}", fontsize=14, fontweight='bold')
            ax.grid(True, alpha=0.3)
            ax.legend(fontsize=12)
            plt.tight_layout()
    
    plt.show()
    
    # Print results
    print("="*80)
    print(f"{IMPULSE_PARAMS[impulse_type]['name']} - Peak: {I_peak/1e3:.1f} kA")
    print("="*80)
    for name, p in results.items():
        print(f"\\n{name.upper().replace('_', ' ')}:")
        print(f"  Peak Current:    {p['I_peak_kA']:.3f} kA")
        print(f"  Specific Energy: {p['W_R_MJ']:.4f} MJ/Ω")
        print(f"  Charge:          {p['Q']:.3f} C")
        print(f"  Max Steepness:   {p['di_dt_max_kA_us']:.2f} kA/µs")
    print("="*80)
    
    return results

print("✓ Calculator ready! Use run_impulse_calculation() below.")

---
## Your Calculation

**Modify parameters below and run:**

In [ ]:
# Customize these parameters for your calculation:

results = run_impulse_calculation(
    impulse_type='PEB',          # Options: 'PEB', 'NEB', 'NFB', '8/20'
    function_type='heidler',     # Options: 'heidler', 'double_exp', 'both', 'damped_sine' (8/20 only)
    I_peak=200e3,                # Peak current in Amperes (e.g., 200e3 = 200 kA)
    duration='infinity',         # 'infinity' for auto, or specify in seconds
    dt=1e-8,                     # Time step (1e-8 = 10 ns)
    show_derivative=True,        # Show di/dt plot?
    compare_functions_flag=False # Compare Heidler vs Double-Exp?
)

---
## Example Library

### Example 1: PEB (10/350 µs) - Standard 200 kA

In [ ]:
# Standard positive first stroke with derivative plot
ex1 = run_impulse_calculation(
    impulse_type='PEB',
    function_type='heidler',
    I_peak=200e3,
    duration='infinity',
    dt=1e-8,
    show_derivative=True
)

### Example 2: Custom Peak Current (282 kA)

In [ ]:
# Custom peak current matching network model
ex2 = run_impulse_calculation(
    impulse_type='PEB',
    function_type='heidler',
    I_peak=282e3,
    duration='infinity',
    dt=1e-8
)

### Example 3: NEB (1/200 µs) - Function Comparison

In [ ]:
# Compare Heidler vs Double-Exponential for negative first stroke
ex3 = run_impulse_calculation(
    impulse_type='NEB',
    function_type='both',
    duration='infinity',
    dt=1e-9,  # 1 ns for faster impulse
    compare_functions_flag=True
)

### Example 4: NFB (0.25/100 µs) - Subsequent Stroke

In [ ]:
# Negative subsequent stroke with both functions
ex4 = run_impulse_calculation(
    impulse_type='NFB',
    function_type='both',
    duration='infinity',
    dt=1e-9,
    compare_functions_flag=True
)

### Example 5: 8/20 µs - Damped Sine Wave

In [ ]:
# Short impulse using damped sine wave (bipolar)
ex5 = run_impulse_calculation(
    impulse_type='8/20',
    function_type='damped_sine',
    I_peak=15e3,  # 15 kA
    duration='infinity',
    dt=1e-8,
    show_derivative=True
)